# DURATION-NEUTRAL DTS-TARGET PORTFOLIO OPTIMIZATION

In [192]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import pyfolio as pf

import warnings
warnings.filterwarnings('ignore')

# Load trading data
data = pd.read_excel("Data/TradingData.xlsx")

# Load treasury data
Treasuries = pd.read_excel("Data/TradingTreasuries.xlsx")

# Load risk-free rate (RF) data
rf_data = pd.read_csv("Data/rf_constant.csv")
rf_data.rename(columns={"Week_Start": "Date"}, inplace=True)

# Format datetime columns
data['Date'] = pd.to_datetime(data['Date'])
data['maturity_date'] = pd.to_datetime(data['maturity_date'], errors='coerce')
data['next_call_date'] = pd.to_datetime(data['next_call_date'], errors='coerce')
Treasuries['Date'] = pd.to_datetime(Treasuries['Date'])
rf_data['Date'] = pd.to_datetime(rf_data['Date'])

# Merge risk-free rates
data = pd.merge(data, rf_data, on='Date', how='left')
data['DTS'] = data['spread'] * data['modified_duration']

# Split data into 3 buckets - low, medium, high DTS subportfolios
bucket_labels = [1, 2, 3]
data['Bucket'] = pd.qcut(data['DTS'], q=len(bucket_labels), labels=bucket_labels, duplicates='drop')

# Filter out extreme spread points - keep middle 99.5% of the data
lower_quantile = data['DTS'].quantile(0.005)
upper_quantile = data['DTS'].quantile(0.995)
data = data[(data['DTS'] > lower_quantile) & (data['DTS'] < upper_quantile)]

# Filter so spread > 30
data = data[data['spread'] > 30]

# Define target DTS by bucket (based on given values)
low_thresh = np.quantile(data['DTS'], 0.33)
mid_thresh = np.quantile(data['DTS'], 0.67)
high_thresh = np.quantile(data['DTS'], 1.0)
target_dts = {1: low_thresh / 2, 2: (low_thresh + mid_thresh) / 2, 3: (mid_thresh + high_thresh) / 2}
print(target_dts)

# Group trading data by Date and Bucket
trading = data.groupby(['Date', 'Bucket']).apply(lambda x: x).reset_index(drop=True)

# Optimizer for Portfolio Weights
def portfolio_optimizer(portfolio, target_dts, initial_weights, previous_weights=None):
    n = len(portfolio)

    def objective(weights):
        excess_returns = portfolio['ytm'] - portfolio['RF']
        risk_adjusted_return = np.dot(weights, excess_returns)

        # Increase penalty for weight concentration
        concentration_penalty = 0.05 * np.sum(weights ** 2)

        # Increase penalty for high portfolio turnover
        if previous_weights is not None and len(previous_weights) == len(weights):
            turnover_penalty = 0.05 * np.sum(np.abs(weights - previous_weights))
        else:
            turnover_penalty = 0

        return -risk_adjusted_return + concentration_penalty + turnover_penalty

    # Constraints for the optimization problem
    constraints = [
        {'type': 'eq', 'fun': lambda w: target_dts - np.dot(w, portfolio['DTS'])},
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Weights must sum to 1
    ]

    # Bounds for weights
    bounds = [(0, 1) for _ in range(n)]

    # Using previous weights or evenly distributed initial weights
    if initial_weights is None or len(initial_weights) != n:
        initial_weights = np.ones(n) / n  # Default equal weights if not given or not matching the portfolio size

    # Run the optimizer
    result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)
    return result.x if result.success else initial_weights  # Return initial_weights if optimization fails


def hedge_duration(portfolio_duration, treasury_data, previous_hedge=None):
    remaining_duration = portfolio_duration
    hedge_results = []
    total_hedge_return = 0  # To accumulate the total return from the hedging instruments

    # Calculate cost per unit of duration for each treasury
    treasury_data['cost_to_duration'] = treasury_data['closing_price'] / treasury_data['modified_duration']
    treasury_data = treasury_data.sort_values(by='cost_to_duration')

    # Track the used quantities from the previous hedge
    if previous_hedge is not None:
        previous_hedge_dict = {treasury['treasury_tool']: treasury['used_quantity'] for _, treasury in previous_hedge.iterrows()}
    else:
        previous_hedge_dict = {}

    for _, treasury in treasury_data.iterrows():
        if abs(remaining_duration) < 1e-6:
            break

        tool_duration = treasury['modified_duration']
        price = treasury['closing_price']
        prev_price = treasury['prev_price']

        # Determine the quantity needed for the current duration
        used_quantity = remaining_duration / tool_duration

        # Adjust the quantity based on the previous day's usage
        prev_quantity = previous_hedge_dict.get(treasury['CUSIP'], 0)
        quantity_change = used_quantity - prev_quantity

        if abs(quantity_change) > 0:
            # Calculate return for the treasury bond
            treasury_return = (price - prev_price) / prev_price if prev_price else 0

            # Calculate the weighted return based on the used quantity change
            weighted_return = quantity_change * treasury_return

            # Append hedge details to the results
            hedge_results.append({
                'treasury_tool': treasury['CUSIP'],
                'used_quantity': quantity_change,
                'cost': quantity_change * price,
                'return': weighted_return  # Add return to the results
            })

        # Update remaining duration to get closer to zero
        remaining_duration -= used_quantity * tool_duration

    # Add total hedge return to the final results
    hedge_results.append({'Total_Hedge_Return': total_hedge_return})

    return pd.DataFrame(hedge_results)

# Calculate Portfolio Metrics
def calculate_portfolio_metrics(sub_data, treasury_hedge):
    coupon_income = (sub_data['Weight'] * sub_data['coupon_rate']).sum()
    capital_gains = (sub_data['Weight'] * (sub_data['closing_price'] - sub_data['prev_price'])).sum()
    hedge_cost = treasury_hedge['cost'].sum() if not treasury_hedge.empty else 0
    hedge_returns = treasury_hedge['return'].sum() if not treasury_hedge.empty else 0
    portfolio_cost = (sub_data['Weight'] * sub_data['closing_price']).sum()

    total_return = coupon_income + capital_gains - hedge_cost + hedge_returns
    excess_return = total_return - (sub_data['RF'] * portfolio_cost).sum()
    portfolio_return = total_return / portfolio_cost

    return {
        'coupon_income': coupon_income,
        'capital_gains': capital_gains,
        'hedge_cost': hedge_cost,
        'portfolio_cost': portfolio_cost,
        'total_return': total_return,
        'excess_return': excess_return,
        'portfolio_return': portfolio_return
    }

def trading_and_hedging(trading_data, treasury_data, target_dts):
    results = []

    # Sort trading data by Date and CUSIP
    trading_data = trading_data.sort_values(by=['CUSIP', 'Date'])
    trading_data['prev_price'] = trading_data.groupby('CUSIP')['closing_price'].shift(1)
    treasury_data['prev_price'] = treasury_data.groupby('CUSIP')['closing_price'].shift(1)

    # Initialize previous weights as None at the beginning
    previous_weights = {}
    previous_hedges = {}

    for date, daily_data in trading_data.groupby('Date'):
        daily_treasury_data = treasury_data[treasury_data['Date'] == date]

        for bucket in [1, 2, 3]:
            sub_data = daily_data[daily_data['Bucket'] == bucket]
            target_dts_value = target_dts.get(bucket, None)

            if target_dts_value is None or len(sub_data) == 0:
                continue

            # Get previous weights or assign equal initial weights
            initial_weights = previous_weights.get(bucket, None)

            # Optimized weights using previous weights as initial guess
            optimized_weights = portfolio_optimizer(sub_data[['DTS', 'ytm', 'closing_price', 'RF']], target_dts_value, initial_weights, previous_weights.get(bucket))

            # Save the current optimized weights for the next iteration
            previous_weights[bucket] = optimized_weights

            sub_data = sub_data.copy()
            sub_data['Weight'] = optimized_weights
            portfolio_duration = (sub_data['Weight'] * sub_data['modified_duration']).sum()

            # Adjust the hedge based on previous hedge positions
            previous_hedge = previous_hedges.get(bucket, None)
            treasury_hedge = hedge_duration(portfolio_duration, daily_treasury_data, previous_hedge)

            # Save the current hedge for the next iteration
            previous_hedges[bucket] = treasury_hedge

            metrics = calculate_portfolio_metrics(sub_data, treasury_hedge)
            metrics['date'] = date
            metrics['bucket'] = bucket
            metrics['rf_rate'] = sub_data['RF'].mean()
            results.append(metrics)

    results_df = pd.DataFrame(results)
    return results_df


# Execute Trading and Hedging
results = trading_and_hedging(trading, Treasuries, target_dts)
print(results)

# Calculate Mean Metrics
total_return = np.sum(results['total_return'])
sharpe_ratio = pf.timeseries.sharpe_ratio(results['total_return'])
print("Total Return:", total_return)
print("Sharpe Ratio:", sharpe_ratio)

{1: 72.64101446353888, 2: 278.17262995104056, 3: 1910.5262516642238}
     coupon_income  capital_gains  hedge_cost  portfolio_cost  total_return  \
0         1.653149       0.000000    1.981598      100.801612     -0.328449   
1         2.320486       0.000000    2.840575       98.425176     -0.520089   
2         5.845315       0.000000   11.236825       93.872128     -5.391510   
3         1.798250       0.071447   -0.089344      100.950150      1.958994   
4         2.202726      -0.217177   -0.132580       98.306263      2.118060   
..             ...            ...         ...             ...           ...   
388       3.538225      -0.034784    7.288565       96.997763     -3.788799   
389       4.124614      -1.811228    3.043216       62.739859     -0.731365   
390       5.719701       0.249288    4.858688       96.495854      1.114274   
391       3.555080       0.043665   -6.144189       97.443642      9.737910   
392       4.125254       1.534066   18.296982       64.512275 

In [194]:
results

,coupon_income,capital_gains,hedge_cost,portfolio_cost,total_return,excess_return,portfolio_return,date,bucket,rf_rate
0,1.653149,0.000000,1.981598,100.801612,-0.328449,-3.251695,-0.003258,2022-05-18,1,0.001
1,2.320486,0.000000,2.840575,98.425176,-0.520089,-5.441348,-0.005284,2022-05-18,2,0.001
2,5.845315,0.000000,11.236825,93.872128,-5.391510,-8.489290,-0.057435,2022-05-18,3,0.001
3,1.798250,0.071447,-0.089344,100.950150,1.958994,-1.271411,0.019406,2022-05-25,1,0.001
4,2.202726,-0.217177,-0.132580,98.306263,2.118060,-1.814191,0.021546,2022-05-25,2,0.001
...,...,...,...,...,...,...,...,...,...,...
388,3.538225,-0.034784,7.288565,96.997763,-3.788799,-79.447054,-0.039061,2024-11-06,2,0.020
389,4.124614,-1.811228,3.043216,62.739859,-0.731365,-40.884875,-0.011657,2024-11-06,3,0.020
390,5.719701,0.249288,4.858688,96.495854,1.114274,-50.993487,0.011547,2024-11-13,1,0.020
391,3.555080,0.043665,-6.144189,97.443642,9.737910,-70.165876,0.099934,2024-11-13,2,0.020
